## Packages

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from tqdm import tqdm
import glob
import os
from time import sleep
import fnmatch


## Load clean data

In [3]:
fomc_rates = pd.read_csv('inputs/Clean_Data/Clean_Rates.csv')
sp500_ret = pd.read_csv('inputs/Clean_Data/Clean_sp500_ret.csv')
stock_tv = pd.read_csv('inputs/Clean_Data/Clean_stock_tv.csv')
Vbilx = pd.read_csv('inputs/Clean_Data/Clean_Vbilx.csv')
Vbirx= pd.read_csv('inputs/Clean_Data/Clean_Vbirx.csv')

## Assemble Data Frames

In [5]:
#rename variables
sp500_ret = sp500_ret.rename(columns = {'date': 'Date','Daily Returns' : 'SP_daily_ret', 'Price': 'SP_daily_price'})
sp500_ret = sp500_ret.rename(columns = {'returns' : 'sp500_ret'})
Vbirx = Vbirx.rename(columns = {'Daily Returns' : 'VBIRX_daily_ret', 'Price': 'VBIRX_daily_price' })
Vbirx = Vbirx.rename(columns = {'Daily Price': 'VBIRX_daily_price' })
Vbilx = Vbilx.rename(columns = {'Daily Price': 'VBILX_daily_price', 'Daily Returns' : 'VBLIX_daily_ret' })

,Date,S&P_500,sp500_ret
0,2015-01-05,2020.58,-0.018278
1,2015-01-06,2002.61,-0.008893
2,2015-01-07,2025.90,0.011630
3,2015-01-08,2062.14,0.017888
4,2015-01-09,2044.81,-0.008404
...,...,...,...
2085,2023-04-18,4154.87,0.000855
2086,2023-04-19,4154.52,-0.000084
2087,2023-04-20,4129.79,-0.005953
2088,2023-04-21,4133.52,0.000903


In [ ]:
stock_tv
stock_tv.drop_duplicates(inplace=True)
stock_tv = stock_tv.rename(columns = {'Value' : 'sp500_tv'})
stock_tv

In [ ]:
final_data = sp500_ret.merge(Vbilx, 
              on=['Date'],
              how='left',
              validate='one_to_one') # or 'many_to_one'

In [ ]:
final_data2 = final_data.merge(Vbirx, on=['Date'],
              how='left',
              validate='one_to_one')

In [ ]:
final_data2

### Create Dummy Variable

In [ ]:
fomc_rates 
fomc_rates['Change'] = np.where(fomc_rates['Increase'] > 0,1,0)
fomc_rates

**Dummy variable equals 1 when the rate increases, 0 if it was decreased.

In [ ]:
#master_df = final_data2.merge(fomc_rates, on = ["Date"], how = 'left', 
 #                     validate = 'm:1', indicator = True)

### Create Event Time Variable

In [ ]:
pd.set_option('display.max_rows',70)


In [ ]:
fomc_rates

In [ ]:
event_time_rets = (
    final_data2.merge(fomc_rates[['Date']], on = ["Date"], how = 'left', 
                      validate = 'm:1', indicator = True)
    
    # create event flag, then use this to create event_id
    .assign(event = lambda x: (x['_merge'] == 'both').astype(int))    
    .assign(event_id = lambda x: x['event'].cumsum(),
            date2 = lambda x: x['Date'])
    
    # reduce dataframe to [-10,+10] around event
    # event id starts 10 days before event and goes to 10 days after
    .assign(event_id = lambda x: x['event_id'].shift(-10))
    .query('event_id > 0')
    .groupby('event_id').head(20)
    
    # helper columns
    .assign(increment =lambda x: np.arange(len(x)),
            inc_at_e = lambda x: x['event']*x['increment'])
    
    #inc_at_e alwasy equal to increment # at firms event
    .assign(inc_at_e = lambda x: x.groupby('event_id')['inc_at_e'].transform(sum))
    
    #compute event time
    .assign(event_time = lambda x: x['increment'] -x['inc_at_e'])
   
    # clear out useless columns
    .drop(['_merge','date2','increment','inc_at_e'],axis=1)
)

In [ ]:
# we need a new var = the date of the event so that we can merge in fomc_rate vars

# Create a new column 'date_when_var2_is_1' and set it to NaN initially
event_time_rets['event_id_date'] = np.nan

# Find the date from 'var1' when 'var2' is equal to 1 for each 'event_id'
date_when_var2_is_1 = event_time_rets[event_time_rets['event'] == 1].groupby('event_id')['Date'].first()

# Iterate through the unique event_ids and set the 'date_when_var2_is_1' value for each event_id
for event_id, date_value in date_when_var2_is_1.items():
    event_time_rets.loc[event_time_rets['event_id'] == event_id, 'event_id_date'] = date_value
    

In [ ]:
# now merge in fomc_rate date
event_time_rets = event_time_rets.merge(fomc_rates,
                      left_on = 'event_id_date',
                      right_on = 'Date',
                      how = 'left',
                      validate = 'm:1').drop('Date_y',axis=1)

In [ ]:
event_time_rets

### Correlation Matrix

In [ ]:
corr_matrix = event_time_rets.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Data with rates')
plt.show()

### Scatter Plots

# Event Time Study Figures

### S&P 500

In [ ]:
# run the regression
model   = smf.ols(formula='sp500_ret ~ C(event_time,Treatment(reference=-1))', data=event_time_rets)
results = model.fit()


# extract the coefficients and confidence intervals
fig, ax            = plt.subplots(figsize=(8, 6))
coef_table         = results.conf_int(alpha=0.05)
coef_table['coef'] = results.params
coef_table.columns = ['lower', 'upper', 'coef']

# the reg table's categorical names stink: C(event_time, Treatment(reference=-1))[T.-4]
# lets rename them to the right level
def categorical_relabel(varname,reference_label):
    try:
        return int(varname.split('[T.')[1].split(']')[0])
    except:
        return reference_label
   
coef_table['event_time'] = coef_table.index.map(lambda x: categorical_relabel(x, -1))

# plot
coef_table.plot(kind='scatter', x='event_time', y='coef', ax=ax, s=100)
ax.errorbar(coef_table['event_time'], coef_table['coef'], yerr=[coef_table['coef'] - coef_table['lower'], coef_table['upper'] - coef_table['coef']], fmt='none', color='black')
plt.title('S&P 500 Returns')
plt.xlabel('Event Time')
plt.ylabel('Coefficient')
ax.axhline(y=0, color='gray', linestyle='--')
plt.show()

### VBILX

In [ ]:
# run the regression
model   = smf.ols(formula='VBLIX_daily_ret ~ C(event_time,Treatment(reference=-1))', data=event_time_rets)
results = model.fit()


# extract the coefficients and confidence intervals
fig, ax            = plt.subplots(figsize=(8, 6))
coef_table         = results.conf_int(alpha=0.05)
coef_table['coef'] = results.params
coef_table.columns = ['lower', 'upper', 'coef']

# the reg table's categorical names stink: C(event_time, Treatment(reference=-1))[T.-4]
# lets rename them to the right level
def categorical_relabel(varname,reference_label):
    try:
        return int(varname.split('[T.')[1].split(']')[0])
    except:
        return reference_label
   
coef_table['event_time'] = coef_table.index.map(lambda x: categorical_relabel(x, -1))

# plot
coef_table.plot(kind='scatter', x='event_time', y='coef', ax=ax, s=100)
ax.errorbar(coef_table['event_time'], coef_table['coef'], yerr=[coef_table['coef'] - coef_table['lower'], coef_table['upper'] - coef_table['coef']], fmt='none', color='black')
plt.title('VBLIX Returns')
plt.xlabel('Event Time')
plt.ylabel('Coefficient')
ax.axhline(y=0, color='gray', linestyle='--')
plt.show()

### VBIRX

In [ ]:

model   = smf.ols(formula='VBIRX_daily_ret ~ C(event_time,Treatment(reference=-1))', data=event_time_rets)
results = model.fit()


# extract the coefficients and confidence intervals
fig, ax            = plt.subplots(figsize=(8, 6))
coef_table         = results.conf_int(alpha=0.05)
coef_table['coef'] = results.params
coef_table.columns = ['lower', 'upper', 'coef']

# the reg table's categorical names stink: C(event_time, Treatment(reference=-1))[T.-4]
# lets rename them to the right level
def categorical_relabel(varname,reference_label):
    try:
        return int(varname.split('[T.')[1].split(']')[0])
    except:
        return reference_label
   
coef_table['event_time'] = coef_table.index.map(lambda x: categorical_relabel(x, -1))

# plot
coef_table.plot(kind='scatter', x='event_time', y='coef', ax=ax, s=100)
ax.errorbar(coef_table['event_time'], coef_table['coef'], yerr=[coef_table['coef'] - coef_table['lower'], coef_table['upper'] - coef_table['coef']], fmt='none', color='black')
plt.title('VBIRX Returns')
plt.xlabel('Event Time')
plt.ylabel('Coefficient')
ax.axhline(y=0, color='gray', linestyle='--')
plt.show()

## Scatterplots

In [ ]:
event_time_rets.columns

### S&P 500

In [ ]:
Sp500_graph = sns.relplot(data = event_time_rets,
                x = "event_time",
               y = "sp500_ret",
                    kind = "line",
               col = "event_id",
                    zorder = 5,
                    hue = "Change",
                col_wrap=2, 
                height=3,
                aspect=1.5, 
                legend=False)

### VBILX

In [ ]:
Sp500_graph = sns.relplot(data = event_time_rets,
                x = "event_time",
               y = "VBLIX_daily_ret",
                    kind = "line",
               col = "event_id",
                    zorder = 5,
                    hue = "Change",
                col_wrap=2, 
                height=3,
                aspect=1.5, 
                legend=False)

### VBIRX

In [ ]:
Sp500_graph = sns.relplot(data = event_time_rets,
                x = "event_time",
               y = "VBIRX_daily_ret",
                    kind = "line",
               col = "event_id",
                    zorder = 5,
                    hue = "Change",
                col_wrap=2, 
                height=3,
                aspect=1.5, 
                legend=False)


# Summary Analysis

## S&P 500

In [ ]:
#Create Dataframe of average SP returns by event
sp_ret_sum = event_time_rets.groupby('event_id_date', as_index=False)['sp500_ret'].mean()
sp_ret_sum.rename(columns = {'sp500_ret': 'sp500_avg_ret'}).head(5)

In [ ]:
#Plot Average Returns
sns.scatterplot(x="event_id_date", 
              y="sp500_ret", 
              data=sp_ret_sum)

plt.title('S&P 500 Average Return by Event', fontsize=18)
plt.xlabel('Date of FOMC Change', fontsize=14)
plt.ylabel('Average Return', fontsize=14)

## VBIRX

In [ ]:
sp_VBIRX_sum = event_time_rets.groupby('event_id_date', as_index=False)['VBIRX_daily_ret'].mean()
VBIRX_ret_sum = sp_VBIRX_sum.rename(columns = {'VBIRX_daily_ret': 'VBIRX_avg_ret'})
VBIRX_ret_sum.head(5)

In [ ]:
sns.scatterplot(x="event_id_date", 
              y="VBIRX_avg_ret", 
              data=VBIRX_ret_sum)

plt.title('VBIRX Average Return by Event', fontsize=18)
plt.xlabel('Date of FOMC Change', fontsize=14)
plt.ylabel('Average Return', fontsize=14)

## VBLIX

In [ ]:
sp_VBLIX_sum = event_time_rets.groupby('event_id_date', as_index=False)['VBLIX_daily_ret'].mean()
VBLIX_ret_sum = sp_VBLIX_sum.rename(columns = {'VBLIX_daily_ret': 'VBLIX_avg_ret'})
VBLIX_ret_sum.head()

In [ ]:
sns.scatterplot(x="event_id_date", 
              y="VBLIX_avg_ret", 
              data=VBLIX_ret_sum)


plt.title('VBLIX Average Return by Event', fontsize=18)
plt.xlabel('Date of FOMC Change', fontsize=14)
plt.ylabel('Average Return', fontsize=14)